<a href="https://colab.research.google.com/github/beaes/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd 'drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
  )

  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
   y_test_norm = np.argmax(y_test, axis=1)

   y_pred_prob = model_trained.predict(X_test)
   y_pred = np.argmax(y_pred_prob, axis=1)

   return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'), 
  ])

In [8]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.6215 - accuracy: 0.2796 - val_loss: 0.9935 - val_accuracy: 0.6973
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.7953 - accuracy: 0.7496 - val_loss: 0.1588 - val_accuracy: 0.9552
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.3747 - accuracy: 0.8855 - val_loss: 0.0840 - val_accuracy: 0.9772
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2386 - accuracy: 0.9284 - val_loss: 0.0450 - val_accuracy: 0.9868
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1926 - accuracy: 0.9422 - val_loss: 0.0317 - val_accuracy: 0.9913


0.9757369614512472

In [9]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1074 - accuracy: 0.9757


0.9757369756698608

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=7,
      verbose=0
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [11]:
space ={
    'batch_size': hp.quniform('batch_size', 50, 200, 20)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 1.2967 - accuracy: 0.6875

 13/138 [=>............................]
 - ETA: 0s - loss: 0.1522 - accuracy: 0.9591

 26/138 [====>.........................]
 - ETA: 0s - loss: 1.0394 - accuracy: 0.9195

 37/138 [=======>......................]
 - ETA: 0s - loss: 0.7370 - accuracy: 0.9417

 45/138 [========>.....................]
 - ETA: 0s - loss: 0.6445 - accuracy: 0.9375

 58/138 [===========>..................]
 - ETA: 0s - loss: 0.5328 - accuracy: 0.9391


In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),
    
    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'), 
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'acuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status':STATUS_OK, 'model':model}

In [14]:
space ={
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.3083858421856321, 'dropout_cnn_block_three': 0.33474519856135215, 'dropout_cnn_block_two': 0.41788606758004154, 'dropout_dense_block_one': 0.35333866300859224, 'dropout_dense_block_two': 0.44079068169569174}
acuracy=0.9530612230300903
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4750125584114768, 'dropout_cnn_block_three': 0.47007297085475663, 'dropout_cnn_block_two': 0.39167163470710664, 'dropout_dense_block_one': 0.5286660440881525, 'dropout_dense_block_two': 0.577774167180894}
acuracy=0.8603174686431885
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.36638419742241474, 'dropout_cnn_block_three': 0.4801521771846162, 'dropout_cnn_block_two': 0.344757085341562, 'dropout_dense_block_one': 0.48139979173989533, 'dropout_dense_block_two': 0.581963632289191}
acuracy=0.9414966106414795
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4839205374226662, 'dropout_cnn_block_three': 0.3170885408638017, 'dropout_cnn_block_two': 0.42242073423697

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'acuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status':STATUS_OK, 'model':model}

In [16]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three', 0, 1),

    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two':hp.uniform('dropout_dense_block_two', 0, 1),
}

best = fmin(
     func_obj,
     space,
     tpe.suggest,
     30,
     Trials()
)
# ten sam kod z innymi parametrami w 'batch_size' daje gorszy wynik

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.1646975043392781, 'dropout_cnn_block_three': 0.37184778964336396, 'dropout_cnn_block_two': 0.7457712038170082, 'dropout_dense_block_one': 0.30364844324588836, 'dropout_dense_block_two': 0.15313040032013636}
acuracy=0.9138321876525879
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.4895437130354645, 'dropout_cnn_block_three': 0.4799621401191222, 'dropout_cnn_block_two': 0.20954904999386792, 'dropout_dense_block_one': 0.7993692000876207, 'dropout_dense_block_two': 0.8054353231335251}
acuracy=0.4095238149166107
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.33685169397172887, 'dropout_cnn_block_three': 0.8552379286145879, 'dropout_cnn_block_two': 0.1876887910324717, 'dropout_dense_block_one': 0.03473725606403533, 'dropout_dense_block_two': 0.14078032415063624}
acuracy=0.7158730030059814
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.5536534239901093, 'dropout_cnn_block_three': 0.8155175705908947, 'dropout_cnn_block_two': 0.598906219396